In [6]:
from moffragmentor import MOF
from pymatgen.core import Structure 


__init__.py: Enabling RDKit 2021.09.4 jupyter extensions


In [7]:
from oximachinerunner import OximachineRunner
runner = OximachineRunner()

In [38]:
m = MOF.from_cif('../tests/test_files/RIDCEN_charged.cif')

In [13]:
f = m.fragment()

In [9]:
def count_radical_electrons(mol):
    radical_electrons = []
    for atom in mol.GetAtoms():
        radical_electrons.append(atom.GetNumRadicalElectrons())
    return sum(radical_electrons)

In [26]:
def get_linker_charges(fragments): 
    linker_charges = []
    
    for linker in fragments.linkers: 
        linker_charges.append(-count_radical_electrons(linker.rdkit_mol))
    
    return linker_charges, sum(linker_charges)

In [11]:
_, lc = get_linker_charges(f)
print(lc)

NameError: name 'f' is not defined

In [27]:
def get_bound_solvent_charges(fragments): 
    mol_charges = []
    
    for solvent in fragments.unbound_solvent: 
        mol_charges.append(-count_radical_electrons(solvent.rdkit_mol))
    
    return mol_charges, sum(mol_charges)

In [13]:
_, bsc = get_bound_solvent_charges(f)
print(bsc)

NameError: name 'f' is not defined

In [21]:
CHARGE_RULES = {
        'O': -2,
        'H' : 1, 
        'F': -1, 
        'Cl': -1, 
        'Br': -1,
        'I': -1
}

In [14]:
def count_charge_in_composition(composition_dict): 
    charge = 0
    for k, v in composition_dict.items():
        charge += CHARGE_RULES[k] * v
        
    return charge

In [15]:
def select_sites(mof, selected_indices): 
    sites = []
    
    for i in selected_indices: 
        sites.append(mof.structure[i])
        
    s = Structure.from_sites(sites)
    
    return s

In [32]:
def get_node_charges(mof, fragments):
    
    node_charges = []
    
    for n in fragments.nodes:
        indices = set(n._original_indices) - n._original_connecting_paths - set(fragments.bound_solvent.indices) - set(mof.metal_indices)
        
        if len(indices) > 0:
            s = select_sites(mof, indices)
            
            charge = count_charge_in_composition(s.composition.as_dict())
            
            node_charges.append(charge) 
    
    return node_charges, sum(node_charges) 

In [33]:
def get_oxidationstates(m):
    res = runner._run_oximachine(m.structure)
    return res['prediction'], sum(res['prediction'])

In [34]:
get_node_charges(m, f)

NameError: name 'f' is not defined

In [42]:
def get_mof_charge(m):
    f = m.fragment()
    _, nc = get_node_charges(m, f)
    _, lc = get_linker_charges(f)
    # _, bsc = get_bound_solvent_charges(f)
    _, mc = get_oxidationstates(m)

    return nc + lc  + mc

In [43]:
get_mof_charge(m)

8.0